In [12]:
import pandas as pd
import numpy as np

In [10]:
# from sklearn.compose import ColumnTransformer
# from sklearn.preprocessing import OneHotEncoder

# ct = ColumnTransformer(transformers=[("encoder", OneHotEncoder(handle_unknown="ignore"), [0, 1])], remainder="passthrough")
# X = np.array(ct.fit_transform(X))
# X_kaggle = np.array(ct.transform(X_kaggle))

In [2]:
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
X_train = X
y_train = y

In [11]:
c = "tipo_vendedor"
v = 1
print("Média:",df_train.loc[df_train[c] == v]["preco"].mean())
print("Mediana:",df_train.loc[df_train[c] == v]["preco"].median())
print("Desvio Padrão:",df_train.loc[df_train[c] == v]["preco"].std())
print("Min:",min(df_train.loc[df_train[c] == v]["preco"]))
print("Max:",max(df_train.loc[df_train[c] == v]["preco"]))
df_train.loc[df_train[c] == v]

In [ ]:
df_train.sort_values(by="preco", ascending=True).iloc[3:-3]

list(df_train).index("quartos")

In [12]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train[:, 84:89] = sc.fit_transform(X_train[:, 84:89])

sc_y = StandardScaler()
y_train = sc_y.fit_transform(y_train.reshape(-1,1))

# sc_y = StandardScaler()
# y_train = sc_y.fit_transform(y_train.reshape(-1,1))

sc_kaggle = StandardScaler()
X_kaggle[:, 83:88] = sc_kaggle.fit_transform(X_kaggle[:, 83:88])

In [4]:
from sklearn.metrics import make_scorer, accuracy_score
from sklearn.model_selection import GridSearchCV
from catboost import CatBoostRegressor

clf = CatBoostRegressor(silent=True)
params = {'depth'         : [6,8,10],
          'learning_rate' : [0.01, 0.05, 0.1],
          'iterations'    : [30, 50, 100]
         }
scorer = make_scorer(accuracy_score)
clf_grid = GridSearchCV(estimator=clf, param_grid=params, scoring=scorer, cv=5)

clf_grid.fit(X_train, y_train)
best_param = clf_grid.best_params_
best_param

C:\Development\Python3\lib\site-packages\sklearn\model_selection\_validation.py:683: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Development\Python3\lib\site-packages\sklearn\model_selection\_validation.py", line 674, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Development\Python3\lib\site-packages\sklearn\metrics\_scorer.py", line 199, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true,
  File "C:\Development\Python3\lib\site-packages\sklearn\metrics\_scorer.py", line 242, in _score
    return self._sign * self._score_func(y_true, y_pred,
  File "C:\Development\Python3\lib\site-packages\sklearn\utils\validation.py", line 63, in inner_f
    return f(*args, **kwargs)
  File "C:\Development\Python3\lib\site-packages\sklearn\metrics\_classification.py", line 202, in accuracy_score
    y_type, y_true, y_pred = _ch

{'depth': 6, 'iterations': 30, 'learning_rate': 0.01}

In [5]:
best_param

{'depth': 6, 'iterations': 30, 'learning_rate': 0.01}

## Utilização de *Gradient Boosting* com a biblioteca *CatBoost* para treinar modelo de regressão

In [16]:
from sklearn.metrics import make_scorer, accuracy_score
from sklearn.model_selection import GridSearchCV
from catboost import CatBoostRegressor

rgs = CatBoostRegressor()
params = {'iterations': [50, 100, 200, 500],
          'depth': [None, 4, 5, 6],
          'l2_leaf_reg': np.logspace(-20, -19, 3),
          'leaf_estimation_iterations': [10],
          'silent': [True],
          'random_seed': [42]
         }

def rmse(actual, predict):
    predict = np.array(predict)
    actual = np.array(actual)
    distance = predict - actual
    square_distance = distance ** 2
    mean_square_distance = square_distance.mean()
    score = np.sqrt(mean_square_distance)
    return score

scorer = make_scorer(rmse, greater_is_better = False)

rgs_grid = GridSearchCV(estimator=clf, param_grid=params, scoring=scorer, cv=10)

rgs_grid.fit(X_train, y_train)

GridSearchCV(cv=10,
             estimator=<catboost.core.CatBoostRegressor object at 0x000001F8ADF13880>,
             param_grid={'depth': [None, 4, 5, 6],
                         'iterations': [50, 100, 200, 500],
                         'l2_leaf_reg': array([1.00000000e-20, 3.16227766e-20, 1.00000000e-19]),
                         'leaf_estimation_iterations': [10],
                         'random_seed': [42], 'silent': [True]},
             scoring=make_scorer(rmse, greater_is_better=False))

In [18]:
best_param = rgs_grid.best_params_
print(best_param)

{'depth': 4, 'iterations': 50, 'l2_leaf_reg': 1e-20, 'leaf_estimation_iterations': 10, 'random_seed': 42, 'silent': True}


In [20]:
regressor = CatBoostRegressor(depth=4, iterations=50, l2_leaf_reg=1e-20, leaf_estimation_iterations=10, random_seed=42, silent=True)
regressor.fit(X_train, y_train)

In [ ]:
# from sklearn.metrics import accuracy_score, confusion_matrix

# y_pred = classifier.predict(X_test)
# acc_sc = accuracy_score(y_test, y_pred)
# cf_mx = confusion_matrix(y_test, y_pred)
# print("Accuracy Score:", acc_sc)
# print(cf_mx)

In [21]:
from sklearn.model_selection import cross_val_score
from datetime import datetime as dt
timestamp = dt.today()

accuracies = cross_val_score(estimator=regressor, X=X_train, y=y_train, cv=10)
print(f"Accuracy: {accuracies.mean()}")
print(f"StdDev: {accuracies.std()}")
try:
    with open("data/cross_val.log", mode="x+") as cvlog:
        cvlog.write("timestamp,accuracy,stddev")
        cvlog.write(f"\n{timestamp},{accuracies.mean()},{accuracies.std()}")
except:
    with open("data/cross_val.log", mode="a") as cvlog:
        cvlog.write(f"\n{timestamp},{accuracies.mean()},{accuracies.std()}")

Accuracy: -1224.6694903075772
StdDev: 935.1971343209372


In [22]:
y_kaggle = sc_y.inverse_transform(regressor.predict(X_kaggle))

In [23]:
with open(f"data/kaggle/kaggle_{timestamp.strftime('%d-%m-%Y_%Hh%M')}.csv", mode="w") as out:
    out.write("Id,preco")
    for i in range(len(y_kaggle)):
        out.write(f"\n{i},{round(y_kaggle[i], 2):.2f}")